# Resistor Combinations

By combining multiple resistors in parallel or series, new resistor values can be achieved.

Here, we evaluate approximations of values with 2 or 3 resistors.

## E12 Series

Resistors are commonly available in standardized sizes.

In [21]:
# here, starting from 1, using just one order of magnitude
e6 = [1.,       1.5,      2.2,      3.3,      4.7,      6.8,      10.]
e12 = [1., 1.2, 1.5, 1.8, 2.2, 2.7, 3.3, 3.9, 4.7, 5.6, 6.8, 8.2, 10.]

## Combinations

In [2]:
import itertools

In [8]:
def pairs(values):
    yield from itertools.combinations_with_replacement(values, 2)

In [37]:
def series(values):
    s = set()
    for (v1, a1, l1), (v2, a2, l2) in pairs(values):
        s.add((v1 + v2, a1 | a2,  f"({l1} -- {l2})"))
    return s

In [38]:
def parallel(values):
    s = set()
    for (v1, a1, l1), (v2, a2, l2) in pairs(values):
        s.add((1.0/(1/v1 + 1/v2), a1 | a2, f"({l1} || {l2})"))
    return s

## Pairs

In [46]:
# base values across 3 orders of magnitude
base = e12
candidates = set(base) | set([10 * v for v in base]) | set([100 * v for v in base])

In [47]:
labeled = [(v, frozenset([v]), f"{v}") for v in candidates]
values = set(labeled)
values.update(series(labeled))
values.update(parallel(labeled))

In [49]:
# see what we have so far
len(values)

1443

## Finding required values

In [102]:
def find(required, tol=0.01, values=values):
    for v in values:
        (value, components, label) = v
        abs_err = abs(required - value)
        rel_err = abs_err / value
        if rel_err <= tol:
            yield v

In [103]:
needed = [100, 103,  113, 141, 215, 439, 1670]

In [104]:
rows = []
for req in needed:
    matches = list(find(req))
    for match in matches:
        prefix = (len(rows), req)
        rows.append(prefix + match)

In [105]:
rows[:4]

[(0, 100, 101.0, frozenset({33.0, 68.0}), '(33.0 -- 68.0)'),
 (1, 100, 101.0, frozenset({1.0, 100.0}), '(1.0 -- 100.0)'),
 (2, 100, 100.0, frozenset({18.0, 82.0}), '(18.0 -- 82.0)'),
 (3, 100, 100.0, frozenset({100.0}), '100.0')]

## Selecting subset of candidates

In [106]:
import mip

In [107]:
m = mip.Model(solver_name=mip.CBC)
m.verbose = 0

In [108]:
# variables to choose a component
choose = {c: m.add_var(var_type=mip.BINARY, name=f"choose_{c}") for c in candidates}

In [109]:
# vars to choose a combination
comb = [m.add_var(var_type=mip.BINARY, name=f"row_{row[0]}") for row in rows]

In [110]:
# constraint: couple combination with component
for row in rows:
    (i, _, _, components, _) = row
    for comp in components:
        m.add_constr(comb[i] <= choose[comp])

In [111]:
# constraint: one combination for each required value
for req in needed:
    matches = [row[0] for row in rows if row[1] == req]
    m.add_constr(mip.xsum([comb[m] for m in matches]) == 1)

In [112]:
# objective: minimize number of components
m.objective = mip.minimize(mip.xsum(choose.values()))

In [113]:
# solve
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [114]:
# extract solution: chosen components
sol = []
for (k,v) in choose.items():
    if v.x > 0.5:
        sol.append(k)
sorted(sol)

[33.0, 47.0, 56.0, 68.0, 180.0, 220.00000000000003, 680.0, 1000.0]

In [115]:
# extract solution: chosen combinations
for row in rows:
    if comb[row[0]].x > 0.5:
        print(row)

(0, 100, 101.0, frozenset({33.0, 68.0}), '(33.0 -- 68.0)')
(4, 103, 103.0, frozenset({56.0, 47.0}), '(47.0 -- 56.0)')
(14, 113, 112.0, frozenset({56.0}), '(56.0 -- 56.0)')
(15, 141, 142.32558139534882, frozenset({680.0, 180.0}), '(680.0 || 180.0)')
(18, 215, 213.0, frozenset({33.0, 180.0}), '(33.0 -- 180.0)')
(21, 439, 440.00000000000006, frozenset({220.00000000000003}), '(220.00000000000003 -- 220.00000000000003)')
(22, 1670, 1680.0, frozenset({680.0, 1000.0}), '(680.0 -- 1000.0)')
